In [110]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [111]:
default_date = datetime.today().strftime('%m/%d/%Y')

In [118]:
def get_date_input():
    date_input = input(f"Please enter a date in the following format MM/DD/YYYY (default is {default_date}): ") or default_date
    try:
        # Try to parse the input date; if invalid, it will raise a ValueError
        valid_date = datetime.strptime(date_input, '%m/%d/%Y').strftime('%m/%d/%Y')
        return valid_date
    except ValueError:
        # If format is incorrect, use the default date
        print("Invalid date format. Using today's date as default.")
        return default_date

# Get the validated date
date = get_date_input()
print("Selected date:", date)

Selected date: 10/01/2024


In [119]:
page = requests.get(f"https://www.yallakora.com/match-center/?date={date}")

In [120]:
def main(page):
    src = page.content
    soup = BeautifulSoup(src, "lxml")
    matches_details = []
    
    championships = soup.find_all("div", {'class': 'matchCard'})
    def get_match_info(championships):
        championship_title = championships.contents[1].find('h2').text.strip()
        all_matches = championships.contents[3].find_all("div", {'class': 'liItem'})
        number_of_matches = len(all_matches)
        for i in range(number_of_matches):
            
            # get team names
            team_A = all_matches[i].find("div", {'class': 'teamA'}).text.strip()
            team_B = all_matches[i].find("div", {'class': 'teamB'}).text.strip()
            
            # get Score
            match_result = all_matches[i].find("div", {'class': 'MResult'}).find_all('span', {'class': 'score'})
            score = f"{match_result[0].text.strip()} - {match_result[1].text.strip()}"
            
            # get match time
            match_time = all_matches[i].find("div", {'class': 'MResult'}).find('span', {'class': 'time'}).text.strip()
            
            # add match_info to matches_details
            matches_details.append({
                'اسم البطولة': championship_title, 
                'الفريق الأول': team_A, 
                'الفريق الثاني': team_B, 
                'موعد المباراة': match_time, 
                'النتيجة': score
                }) 
            
    for i in range (len(championships)):
        get_match_info(championships[i])  
        
    keys = matches_details[0].keys()
    
    with open ('E:/Data analysis/DEPI/11/matches-details.csv', 'w', encoding='utf-8', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(matches_details)
        print('file_Created')
      
main(page)

file_Created
